In [151]:
module cse

using DataStructures: OrderedDict

immutable CacheElement
    name::Symbol
    expr::Nullable{Expr}
end

typealias Cache OrderedDict{Symbol, CacheElement}

cacheify!(setup, expr) = expr

function cacheify!(setup::Cache, expr::Expr)
    if expr.head == :call
        takes_one_arg = length(expr.args) == 2
        function_name = expr.args[1]
        arg_name = expr.args[2]
        uses_avialable_arg = any(arg_name == s.name for s in values(setup))
        can_be_cached = takes_one_arg && uses_avialable_arg
        if can_be_cached
            cached_name = Symbol(function_name, arg_name)
            if !haskey(setup, cached_name)
                cached_varname = gensym(cached_name)
                    setup[cached_name] = CacheElement(cached_varname, :($cached_varname = $(copy(expr))))
            else
                cached_varname = setup[cached_name].name
            end
            return cached_varname
        end
    end
          
    for (i, child) in enumerate(expr.args)
        expr.args[i] = cacheify!(setup, child)
    end
    expr
end
    
    disqualify_symbols!(available, disqualified, expr) = nothing
    disqualify_symbols!(available, disqualified, expr::Symbol) = push!(disqualified, expr)
    
    
    function disqualify_symbols!(available, disqualified, expr::Expr)
        for arg in expr.args
            disqualify_symbols!(available, disqualified, arg)
        end
    end
    
    available_leaf_symbols!(available, disqualified, expr) = nothing
    available_leaf_symbols!(available, disqualified, expr::Symbol) = push!(available, expr)

    function available_leaf_symbols!(available, disqualified, expr::Expr)
        if expr.head == :line
            # nothing
        elseif expr.head == :(=)
            disqualify_symbols!(available, disqualified, expr.args[1])
            for arg in expr.args[2:end]
                available_leaf_symbols!(available, disqualified, arg)
            end
        else
            for arg in expr.args
                available_leaf_symbols!(available, disqualified, arg)
            end
        end
    end
        
    function available_leaf_symbols(expr)
        available = Set{Symbol}()
        disqualified = Set{Symbol}()
        available_leaf_symbols!(available, disqualified, expr)
        setdiff!(available, disqualified)
    end
    
function cacheify(expr::Expr)
    setup = Cache()
        for var in available_leaf_symbols(expr)
            setup[var] = CacheElement(var, nothing)
        end
    while true
        num_setup = length(setup)
        expr = copy(expr)
        expr = cacheify!(setup, expr)
        if length(setup) == num_setup
            break
        end
    end
        Expr(:block, [get(v.expr) for v in values(setup) if !isnull(v.expr)]..., expr)
end

    macro cse(expr)
        result = cacheify(expr)
        println(result)
        esc(result)
    end
    

    
end

cse

In [154]:
function g(x)
    println("g")
    1
end

function f(x)
    println("f")
    2
end

x = 1

expr = quote
    [f(g(x)) == i for i in 1:5]
end

quote  # In[154], line 14:
    [f(g(x)) == i for i = 1:5]
end

In [155]:
eval(cse.cacheify(expr))

g
f


5-element Array{Bool,1}:
 false
  true
 false
 false
 false

In [157]:
@cse.cse begin
    [f(g(x)) == i for i in 1:5]
end

begin 
    ##gx#335 = g(x)
    ##f##gx#335#336 = f(##gx#335)
    begin  # In[157], line 2:
        [##f##gx#335#336 == i for i = 1:5]
    end
end
g
f


5-element Array{Bool,1}:
 false
  true
 false
 false
 false